In [1]:
import lightgbm as lgb
import pandas as pd
import math
import numpy as np
import pickle

from sklearn.model_selection import train_test_split

df = pd.read_csv('../data/features.csv')

target = df['target']
features = df.drop('target', axis=1)

In [2]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'mae',
    'learning_rate': 0.001
}

x_train, x_valid, y_train, y_valid = train_test_split(features, target, test_size=0.2)

d_train = lgb.Dataset(x_train, label=y_train)
d_valid = lgb.Dataset(x_valid, label=y_valid)
n_estimators = 10000
watchlist = [d_valid ]
estimator = lgb.train(params, d_train, n_estimators, watchlist, verbose_eval=10, early_stopping_rounds=100)

pickle.dump(estimator, open('../data/estimator.lgb', 'wb'))

Training until validation scores don't improve for 100 rounds.
[10]	valid_0's l1: 5.99345
[20]	valid_0's l1: 5.99345
[30]	valid_0's l1: 5.99345
[40]	valid_0's l1: 5.99345
[50]	valid_0's l1: 5.99345
[60]	valid_0's l1: 5.99345
[70]	valid_0's l1: 5.99345
[80]	valid_0's l1: 5.99345
[90]	valid_0's l1: 5.99345
[100]	valid_0's l1: 5.99345
Early stopping, best iteration is:
[1]	valid_0's l1: 5.99345


In [3]:
df_test = pd.read_csv('../data/test_features.csv')
pred_all = []

for segid in np.unique(df_test['segid']):
    df_segid = df_test[df_test['segid'] == segid].drop('segid', axis=1)
    predictions = []
    for i in range(df_segid.shape[0]):
        x = df_segid.iloc[i, :]
        pred = estimator.predict(x)
        predictions.append(pred)
    pred_all.append({'seg_id': segid, 'time_to_failure': np.mean(predictions)})
pred_df = pd.DataFrame(pred_all)
pred_df.to_csv('prediction.csv', index=False)

C:\Users\lutz.kuenneke\AppData\Roaming\Python\Python37\site-packages\lightgbm\basic.py:478: UserWarning: Converting data to scipy sparse matrix.
  warnings.warn('Converting data to scipy sparse matrix.')
